In [1]:
import os
import shutil
import numpy as np
import time
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image as imgprocessor
import glob
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
path = './trained_models/FINAL_MODELALL_acc0.9825_ep26.h5'
model = load_model(path)

In [15]:
classes = {0:'NoTumor', 1:'Tumor'}
score_min_thresh = 0.8

def preprocess(imgpath):
    img = imgprocessor.load_img(imgpath, target_size=(256,256), color_mode='grayscale')
    img = imgprocessor.img_to_array(img)
    img /= 255.
    return img

def infer(imgbatch):
    outstatus = [0]*len(imgbatch)
    t1 = time.time()
    out = model.predict(imgbatch)
    t2 = time.time()
    #print(f'Forward Time:{t2-t1:.3f}')
    index = np.argmax(out,axis=1)
    for i in range(len(index)):
        if index[i] == 1 and out[i][index[i]] >= score_min_thresh:
            outstatus[i] = 1
            
    return outstatus


In [21]:
base_root = 'DATA/'
out_base = 'DATATUMORONLY'
os.makedirs(out_base, exist_ok=True)
bs = 8
global_mods = ['flair','t2w','t1wce']
global_settye = ['train']

avg_infer = 0
avg_prep = 0
for root,dirs,files in os.walk(base_root):
    root = root.replace('\\','/')
    items = root.split('/')
    if len(items)>=4:
        settype = items[1].lower()
        if settype not in global_settye:
            #print('Continue settype',settype)
            continue
        patid = items[2]
        mod = items[3].lower()
        if mod not in global_mods:
            #print('Continue mod',mod)
            continue
        outdir = os.path.join(out_base, settype, patid, mod)
        os.makedirs(outdir, exist_ok=True)
        div, _ = divmod(len(files), bs)
        dset = tqdm(range(0, len(files), bs), total=(div+1))
        for i in dset:
            if i%10 == 0:
                dset.set_description(f"Avg Infer: {avg_infer/max(i,1):.3f} | Avg Prep: {avg_prep/max(i,1):.3f}")
            batch_files = files[i:i+bs]
            new_batch_files = []
            for file in batch_files:
                if not os.path.isfile(os.path.join(outdir, file)):
                    new_batch_files.append(file)
                else:
                    #print('Path Exist:',os.path.join(outdir, file))
                    pass
                    
            if len(new_batch_files) == 0:
                #print('Empty Batch....')
                continue
                
            batch_files = new_batch_files
            new_batch_files = None
            batch_paths = [os.path.join(root, file) for file in batch_files]
            
            t1 = time.time()
            batch_images = [preprocess(imgpath) for imgpath in batch_paths]
            t2 = time.time()
            #print(f'Preprocess Time:{t2-t1:.3f}')
            avg_prep += t2-t1
            
            batch_images = np.array(batch_images)
            
            t1 = time.time()
            outstatus = infer(batch_images)
            t2 = time.time()
            #print(f'Infer Time:{t2-t1:.3f} | Batch:{batch_images.shape}')
            avg_infer += t2-t1
            
            for j,status in enumerate(outstatus):
                if status:
                    shutil.copyfile(batch_paths[j], os.path.join(outdir, batch_files[j]))

Avg Infer: 0.000 | Avg Prep: 0.000:   3%|███▋                                                                                                                                      | 1/37 [00:00<00:10,  3.53it/s]

Infer Time:0.248 | Batch:(8, 256, 256, 1)


Avg Infer: 0.000 | Avg Prep: 0.000:   5%|███████▍                                                                                                                                  | 2/37 [00:00<00:10,  3.45it/s]

Infer Time:0.276 | Batch:(8, 256, 256, 1)


Avg Infer: 0.000 | Avg Prep: 0.000:   8%|███████████▏                                                                                                                              | 3/37 [00:00<00:09,  3.56it/s]

...

Avg Infer: 0.030 | Avg Prep: 0.004:  38%|███████████████████████████████████████████████████▊                                                                                     | 14/37 [00:03<00:06,  3.60it/s]

Infer Time:0.240 | Batch:(8, 256, 256, 1)


Avg Infer: 0.019 | Avg Prep: 0.002:  73%|███████████████████████████████████████████████████████████████████████████████████████████████████▉                                     | 27/37 [00:04<00:01,  6.28it/s]

Infer Time:0.272 | Batch:(8, 256, 256, 1)
Infer Time:0.082 | Batch:(1, 256, 256, 1)


KeyboardInterrupt: 